# Sensitivity Analysis: Combined SPECT/CT + Sestamibi

### Objective

Calculate the **sensitivity of combined SPECT/CT + Sestamibi imaging**, assessing true positive rates among patients with confirmed disease.

### Data Source

- `side_concordance_results.xlsx` (pre-operative imaging and surgical outcomes)

### Outputs

- **Sensitivity for SPECT/CT + Sestamibi scans:** **71.8%**
  - **True Positives:** 270
  - **Total with disease:** 376

---


In [1]:
import pandas as pd 
preop_imaging_df = pd.read_excel('../../Data/side_concordance_results.xlsx')

In [2]:
def extract_side(location):
    if pd.isnull(location):
        return 'No Scan'
    
    if 'Left' in location:
        return 'Left'
    elif 'Right' in location:
        return 'Right'
    else:
        return 'Unknown'
    
preop_imaging_df['Surgical Side'] = preop_imaging_df['Surgical_Findings'].apply(extract_side)

In [3]:
def check_spect_sestamibi(row):
    spect = row['SPECT/CT']
    sestamibi = row['Sestamibi']
    
    spect_localizing = pd.notnull(spect) and spect not in ['No Scan', 'Non-localizing']
    sestamibi_localizing = pd.notnull(sestamibi) and sestamibi not in ['No Scan', 'Non-localizing']
    
    if not pd.notnull(spect) and not pd.notnull(sestamibi):
        return 'No SPECT/CT or Sestamibi Recorded'
    elif spect_localizing and sestamibi_localizing:
        return f"SPECT: {spect} | Sestamibi: {sestamibi}"
    elif spect_localizing:
        return f'SPECT/CT: {spect}'
    elif sestamibi_localizing:
        return f'Sestamibi: {sestamibi}'
    else:
        return 'Non-localizing'
    
preop_imaging_df['SPECT/CT-Sestamibi'] = preop_imaging_df.apply(check_spect_sestamibi, axis=1)
preop_imaging_df['SPECT/CT-Sestamibi'].head()

0    Sestamibi: Right Inferior
1               Non-localizing
2     Sestamibi: Left Inferior
3     Sestamibi: Left Inferior
4              Sestamibi: Left
Name: SPECT/CT-Sestamibi, dtype: object

In [5]:
def check_concordance(row):
    imaging = row['SPECT/CT-Sestamibi']
    surgical = row['Surgical_Findings']
    
    if pd.isnull(imaging) or pd.isnull(surgical):
        return 'Unknown'
    imaging_glands = [g.strip() for g in imaging.split(',')]
    surgical_glands = [g.strip() for g in surgical.split(',')]
    
    if any(sg in imaging_glands for sg in surgical_glands):
        return 'Yes (Exact)'
    
    imaging_sides = []
    for ig in imaging_glands:
        if 'Left' in ig:
            imaging_sides.append('Left')
        if 'Right' in ig:
            imaging_sides.append('Right')
    
    surgical_sides = []
    for sg in surgical_glands:
        if 'Left' in sg:
            surgical_sides.append('Left')
        if 'Right' in sg:
            surgical_sides.append('Right')
    
    if any(side in surgical_sides for side in imaging_sides):
        return 'Yes (Side)'
    
    return 'Incorrect Localization'


In [8]:
# Define modality
modality = 'SPECT/CT-Sestamibi'

# Apply extract_side function if not already done to get imaging sides
preop_imaging_df['SPECT/CT-Sestamibi Imaging Side'] = preop_imaging_df[modality].apply(extract_side)

# Filter to scans that localized anywhere (Left, Right)
localized_spectmibi = preop_imaging_df[preop_imaging_df['SPECT/CT-Sestamibi Imaging Side'].isin(['Left', 'Right'])].copy()
nonlocalized_spectmibi = preop_imaging_df[preop_imaging_df[modality] == 'Non-localizing'].copy()
# total localized and non-localized sestamibi
localized_spectmibi['Calculated Concordance'] =localized_spectmibi.apply(check_concordance, axis=1)


true_positives = localized_spectmibi['Calculated Concordance'].isin(['Yes (Exact)', 'Yes (Side)']).sum()
false_negatives = nonlocalized_spectmibi.shape[0]
false_negatives 
nonlocalized_spectmibi

,Procedure_Date,Patient_ID,Age,Gender,Surgical_Findings,Surgical Cure,SPECT/CT,Ultrasound,4D CT Scan,Sestamibi,MRI,Ultrasound_Concordance,SPECT/CT_Concordance,4D CT Scan_Concordance,Sestamibi_Concordance,MRI_Concordance,Surgical Side,SPECT/CT-Sestamibi,SPECT/CT-Sestamibi Imaging Side
1,2012-05-08,182059,70,F,Left Inferior,Yes,NaN,Non-localizing,NaN,Non-localizing,Right Inferior,No,No Scan,No Scan,No,No,Left,Non-localizing,Unknown
9,2012-09-18,183702,54,M,Right Superior,Yes,NaN,Right,NaN,Non-localizing,Right,Yes (Side),No Scan,No Scan,No,Yes (Side),Right,Non-localizing,Unknown
15,2013-01-30,189853,57,F,Right Inferior,Yes,NaN,Non-localizing,NaN,Non-localizing,NaN,No,No Scan,No Scan,No,No Scan,Right,Non-localizing,Unknown
17,2013-03-01,164212,57,F,Not Surgically Cured,No,Non-localizing,Non-localizing,NaN,Non-localizing,NaN,Yes (Side),Yes (Side),No Scan,Yes (Side),No Scan,Unknown,Non-localizing,Unknown
18,2013-04-03,191724,62,F,Right Superior,Yes,NaN,Non-localizing,NaN,Non-localizing,Non-localizing,No,No Scan,No Scan,No,No,Right,Non-localizing,Unknown
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
367,2024-09-11,187958,,F,Right Inferior,Yes,NaN,NaN,NaN,Non-localizing,Right Inferior,No Scan,No Scan,No Scan,No,Yes (Exact),Right,Non-localizing,Unknown
377,2024-12-24,156604,67,F,4 Gland Hyperplasia,Yes,Non-localizing,NaN,Right Inferior,NaN,NaN,No Scan,Yes (Side),No,No Scan,No Scan,Unknown,Non-localizing,Unknown
382,2025-01-21,307537,50,F,Right Inferior,Yes,Non-localizing,Non-localizing,Non-localizing,NaN,NaN,No,No,No,No Scan,No Scan,Right,Non-localizing,Unknown
395,2025-04-01,311292,80,F,"Right Superior, Right Inferior",Yes,Non-localizing,NaN,Left Inferior,NaN,NaN,No Scan,No,No,No Scan,No Scan,Right,Non-localizing,Unknown


In [9]:
sensitivity = true_positives / (true_positives + false_negatives)
print(f"SPECT/CT + Sestamibi Sensitivity: {sensitivity:.2%} ({true_positives} True Positive / {true_positives + false_negatives} total with disease)")

SPECT/CT + Sestamibi Sensitivity: 71.81% (270 True Positive / 376 total with disease)
